# Target Offloading

## Jupyter Notebooks

The material of this course is provided mainly in the form of Jupyter Notebooks.
In case you are not familiar with them:
* Notebooks are comprised of **cells**.
* Important cell types are
    * **Text cells** which are written using [Markdown](https://www.markdownguide.org/) - double click this text to see it.
    * **Code cells** which contain Python code to be executed.
* Cells can be executed with
    * the play button above,
    * via the menu,
    * by typing *ctrl + return* to execute and stay with the current cell, or
    * with *shift + return* to execute and move to the next cell.

In [ ]:
print('Hello world from a Python cell')

Apart from executing Python code, which is not really useful for *this* tutorial, code cells can also execute terminal commands by prefixing them with `!`.
The below cell calls `nvidia-smi`, the *Nvidia System Management Interface*, that reports real-time information about installed Nvidia GPU devices including
* utilization,
* memory usage,
* temperature, and
* running processes.

In [ ]:
!nvidia-smi

## ICE Magic

The last way to use code cells that is relevant for this tutorial are **magic commands**.
The below cell loads an extension called **ICE**, short for instantiate-compile-execute.

In [ ]:
%load_ext ice.magic

After the extension is loaded, we can use the `%%cpp` magic command.
It takes the remaining contents of the cell it is included in and does the following steps:
* *Instantiate*: The code provided is augmented with auxiliary boilerplate code such as standard includes and written to file.
* *Compile*: The generated file is compiled to a binary.
* *Execute*: The binary is executed.

We start with a simple hello world example to illustrate how ICE can be used.

In [ ]:
%%cpp
  👆

printf("Hello world from a compiled application\n");

Our magic function also takes additional parameters which can be displayed with the following cell.

In [ ]:
%%cpp --help
//#

Let's specify the (code) output file as well as add the *verbose* and *display* flags.

In [ ]:
%%cpp -o code/hello-world.cpp -v -d
                              👆 👆

printf("Hello world from a compiled application\n");

## GPU Programming Overview

We start by reviewing a short presentation.
Execute the cell below to display it or download and open it locally.

In [ ]:
%%HTML
<div align="center"><iframe src="slides/PPHPS-GPU.pdf" width=800 height=500 frameborder=0></iframe></div>


In contrast to other GPU programming approaches, OpenMP follows a slightly different naming convention ([OpenMP 5.1 - 1.2.1](https://www.openmp.org/spec-html/5.1/openmpsu1.html)).
* A *device* is 'an implementation-defined logical execution engine'.
* The *host*, or *initial device*, is the device on which the OpenMP program begins execution.
* The *target* is a device onto which code and data may be offloaded from the host device. In many cases it is a GPU, but it doesn't have to be.

## OpenMP Target

Before we dive deeper, make sure to load the ICE magic commands with the below cell.
This allows executing all following code examples.

In [ ]:
%load_ext ice.magic

The `target` construct transfers execution to the device (target) ([OpenMP 5.1 - 2.14.5](https://www.openmp.org/spec-html/5.1/openmpsu68.html)).
\
Everything within the target block is executed on the target.
Only a sub-set of features is available, e.g. no `std::cout`. If you are familiar with other GPU programming approaches -- similar restrictions apply with OpenMP as well.

In [ ]:
%%cpp_omp_target -o code/hello-target.cpp

std::cout << "Hello from the CPU" << std::endl;

#pragma omp target
{           👆
    printf("Hello from the GPU\n");
} //# implicit synchronization with the target

### Compilation

You might have noticed that we used a different magic than discussed in the introduction -- `cpp_omp_target`.
\
This switches to a different compiler, `nvc++`, and adds a different set of flags.
As before, using the `-v` switch displays more detailed information.
\
Adding the optional `-Minfo=mp` compiler flag triggers the compiler to emit information about how the application is mapped to the target.

In [ ]:
%%cpp_omp_target -o code/hello-target.cpp -v -f Minfo=mp

std::cout << "Hello from the CPU" << std::endl;

#pragma omp target
{
    printf("Hello from the GPU\n");
} //# implicit synchronization with the target

### Checking for Host Execution

In some cases it might be necessary to programmatically check whether execution is currently on the host or on the device, e.g. when using multiple nested functions.
`omp_is_initial_device` can be used to perform that check ([OpenMP 5.1 - 3.7.6](https://www.openmp.org/spec-html/5.1/openmpsu166.html)).

In [ ]:
%%cpp_omp_target -o code/initial-device.cpp

std::cout << omp_is_initial_device() << std::endl;
             👆

#pragma omp target
{
    printf("%d\n", omp_is_initial_device());
                   👆
} 

## Parallel Execution

So far, everything in our `target` region has been executed serially since the target construct doesn't generate parallelism.
\
In the following steps, we will add hierarchical parallelism (to match the GPU architecture discussed before), and workload sharing.
\
We will use the the following example as baseline.
In it, the loop is executed on the device, but only with a single thread.

In [ ]:
%%cpp_omp_target -o code/target-serial.cpp

#pragma omp target
for (auto i = 0; i < 10; ++i)
    printf("%d\n", i);

### Teams

`teams` construct generates a *league of teams* ([OpenMP 5.1 - 2.7](https://www.openmp.org/spec-html/5.1/openmpse15.html)).
\
A team is comparable but not necessarily identical to a CUDA thread block.
Each team initially has only one thread and each team executes the same code.
The number of teams can be *limited* with `num_teams` -- this sets an upper bound, not the exact number.

The id of the current team can be querried with `omp_get_team_num` ([OpenMP 5.1 - 3.4.2](https://www.openmp.org/spec-html/5.1/openmpsu152.html)).
\
`omp_get_thread_num` returns the current thread id *within the current team*.

In [ ]:
%%cpp_omp_target -o code/target-teams.cpp

#pragma omp target
#pragma omp teams num_teams(2)
            👆   👆
    printf("Team %d, thread %d\n", omp_get_team_num(), omp_get_thread_num());
                                   👆

### Parallel

`parallel` generates a parallel region with multiple threads per team.
The number of threads per team can be limited with `thread_limit`

In [ ]:
%%cpp_omp_target -o code/thread-limit.cpp

#pragma omp target
#pragma omp teams parallel num_teams(2) thread_limit(2)
                  👆                    👆
    printf("Team %d, thread %d\n", omp_get_team_num(), omp_get_thread_num());

### Distribute

For loops, worksharing constructs are required additionally.

`distribute` distributes the iteration space across teams ([OpenMP 5.1 - 2.11.6](https://www.openmp.org/spec-html/5.1/openmpsu50.html)).
\
Schedules can be specified using `dist_schedule`.

In [ ]:
%%cpp_omp_target -o code/target-distribute.cpp

#pragma omp target
#pragma omp teams num_teams(2)
#pragma omp distribute
            👆
for (auto i = 0; i < 10; ++i)
    printf("Team %d, thread %d, i = %d\n", omp_get_team_num(), omp_get_thread_num(), i);

### For

`for` distributes the *team's* iteration space over the team's threads.

In [ ]:
%%cpp_omp_target -o code/target-for.cpp

#pragma omp target
#pragma omp teams num_teams(2)
#pragma omp distribute parallel for
                                👆
for (auto i = 0; i < 10; ++i)
    printf("Team %d, thread %d, i = %d\n", omp_get_team_num(), omp_get_thread_num(), i);

### SIMD

Additionally, the `simd` construct is also available.
What exactly is mapped how is compiler dependent.
For NVIDIA, *usually* teams are mapped to CUDA thread blocks, threads are mapped to CUDA threads and simd is ignored.

In [ ]:
%%cpp_omp_target -o code/target-simd.cpp

#pragma omp target teams distribute parallel for simd
                                                 👆
for (auto i = 0; i < 10; ++i)
    printf("Team %d, thread %d, i = %d\n", omp_get_team_num(), omp_get_thread_num(), i);

### Loop

Since OpenMP 5, the alternative `target teams loop` construct is available, which serves as a shorthand.
Since OpenMP 6, there is also `target loop`.

In [ ]:
%%cpp_omp_target -o code/target-loop.cpp

#pragma omp target teams loop
                    👆   👆
for (auto i = 0; i < 10; ++i)
    printf("Team %d, thread %d, i = %d\n", omp_get_team_num(), omp_get_thread_num(), i);

## An Aside on Managed Memory

On newer architecture, managed memory or unified memory is available (see, e.g., this [blog post](https://developer.nvidia.com/blog/unified-memory-cuda-beginners/)).
When used, allocations are done as managed memory and all transfers between host and target are done implicitly.
We will start with this version since it requires less coding effort and is more robust for most use cases when beginning with GPU programming.

~~In OpenMP it is activated by adding the compiler flag `-gpu=managed` and by specifying `#pragma omp requires unified_shared_memory`.~~
\
In newer versions of the NVC++ compiler, the flag is now `-gpu=mem:managed`.
Additionally, `#pragma omp requires unified_shared_memory` now requires `-gpu=mem:unified` and is only available on systems that support full CUDA Unified Memory capability (e.g. Nvidia Grace Hopper).

In [ ]:
%%cpp_omp_target -o code/managed-mem.cpp -v -f gpu=mem:managed
                                                   👆

//# #pragma omp requires unified_shared_memory
👆

int *vec = new int[10];
for (auto i = 0; i < 10; ++i)
    vec[i] = i;

#pragma omp target teams distribute parallel for // no map clauses
for (auto i = 0; i < 10; ++i)
    vec[i] *= 2;

for (auto i = 0; i < 10; ++i)
    std::cout << vec[i] << " ";
std::cout << std::endl;

free(vec);

## Exercise: Stream Benchmark

The stream benchmark application can be used to assess bandwidth.
\
It copies data between two arrays in a ping-pong fashion, each time increasing each element by one.
The latter allows for checking correctness of the implemented operation.

The serial baseline code is available at [code/exercise/stream.cpp](code/exercise/stream.cpp).
It can be compiled and executed using the following cells.

In [ ]:
!g++ -O3 -march=native -std=c++17 -Wall -o code/exercise/stream code/exercise/stream.cpp

In [ ]:
!code/exercise/stream

Optionally, the default parameters can be overwritten with `./stream num_elements num_repetitions`

In [ ]:
!code/exercise/stream $((1024 * 1024)) 2

Your task is to refactor the application to use OpenMP target offloading to accelerate the main kernel.
Use managed memory for this version.
\
You can compile and execute the application with the cells below after addressing the remaining TODOs.

In [ ]:
!(TODO:compiler) (TODO:general-flags) (TODO:omp-flags) (TODO:managed-mem) -o code/exercise/stream code/exercise/stream.cpp

In [ ]:
!code/exercise/stream $((64 * 1024 * 1024)) 4

After you have implemented a correct version (`Stream check completed` is printed at the end), evaluate performance for different configurations:

* Is the estimated bandwidth constant for all iterations?
* Does the bandwidth depend on the problem size (number of elements)?
* Does specifying/ changing `num_teams` and `thread_limit` affect the performance?

Remember that you can get additional information about parallelization applied by the compiler with `-Minfo=mp`.
Additionally setting the environment variable `NVCOMPILER_ACC_NOTIFY=1` when executing will print execution configuration information for every kernel executed. 

### Possible Solution

A possible solution is given in [code/solution/stream.cpp](code/solution/stream.cpp).
It can be compiled and executed using the following cells.

In [ ]:
!nvc++ -O3 -std=c++17 -mp=gpu -gpu=mem:managed -Minfo=mp -o code/solution/stream code/solution/stream.cpp

In [ ]:
!NVCOMPILER_ACC_NOTIFY=1 code/solution/stream $((64 * 1024 * 1024)) 4

## Target Data

Managed memory is a powerful tool, but can have negative effects on performance, as seen in the previous exercise.
The main alternative is taking control over memory placement, allocation and movement.
OpenMP supports this through `target data`.

Each target region spans its own data environment.
In addition to the clauses already discussed in the OpenMP for CPU part, target data mapping clauses are available.
Let's consider the following example.

In [ ]:
%%cpp_omp_target -o code/target-map.cpp

int *vec = new int[10];
for (auto i = 0; i < 10; ++i)
    vec[i] = i;

#pragma omp target teams distribute parallel for map(tofrom: vec[0:10])
                                                 👆
for (auto i = 0; i < 10; ++i)
    vec[i] *= 2;

for (auto i = 0; i < 10; ++i)
    std::cout << vec[i] << " ";
std::cout << std::endl;

free(vec);

Here, the contents of `vec` are copied to the target when entering the target region and copied back to the host when leaving it.
Available map types in the `map` clause are
* `to` which copies data from host to target,
* `from` which copies data from target to host,
* `tofrom` which combines the behavior of to and from, and
* `alloc` which allocates data on the target but does not initialize it.

### Implicit Behavior

Target data environments implement the following implicit behavior if not specified otherwise:
* Scalar variables are `firstprivate`
    * They are copied to the device and each thread has its own version
    * Changes are neither synchronized between threads nor copied back to the host

In [ ]:
%%cpp_omp_target -o code/implicit-data.cpp

int a = 10;
   👆
static int b = 20;
          👆

#pragma omp target teams parallel
if (0 == omp_get_team_num() && 0 == omp_get_thread_num()) {
    printf("a = %d, b = %d\n", a, b);
    a *= 10;
    b *= 10;
}

printf("a = %d, b = %d\n", a, b);

* Statically allocated arrays are treated as `map(tofrom)`

In [ ]:
%%cpp_omp_target -o code/static-array.cpp

int vec[] = {0, 1, 2, 3, 4, 5, 6, 7, 8, 9};
    👆

#pragma omp target teams distribute parallel for
for (auto i = 0; i < 10; ++i)
    vec[i] *= 2;

for (const auto& val : vec)
    std::cout << val << " ";
std::cout << std::endl;

* Dynamically allocated arrays need to be mapped explicitly.

In [ ]:
%%cpp_omp_target -o code/missing-map.cpp

int *vec = new int[10];
    👆
for (auto i = 0; i < 10; ++i)
    vec[i] = i;

#pragma omp target teams distribute parallel for
//# due to the missing map clause, this example will not work
for (auto i = 0; i < 10; ++i)
    vec[i] *= 2;

for (auto i = 0; i < 10; ++i)
    std::cout << vec[i] << " ";
std::cout << std::endl;

free(vec);

### Target Data Region

Mapping data for each target region individually can generate some drawbacks (code bloating, performance issues).
OpenMP offers `target data` constructs as an alternative ([OpenMP 5.1 - 2.14.2](https://www.openmp.org/spec-html/5.1/openmpsu65.html)).

In [ ]:
%%cpp_omp_target -o code/target-data.cpp

int *vec = new int[10];
for (auto i = 0; i < 10; ++i)
    vec[i] = i;

#pragma omp target data map(tofrom: vec[0:10])
{                 👆
    #pragma omp target teams distribute parallel for
    for (auto i = 0; i < 10; ++i)
        vec[i] *= 2;

    #pragma omp target teams distribute parallel for
    for (auto i = 0; i < 10; ++i)
        vec[i] *= 2;
}

for (auto i = 0; i < 10; ++i)
    std::cout << vec[i] << " ";
std::cout << std::endl;

free(vec);

If the software architecture requires a more unstructured approach `target enter data` ([OpenMP 5.1 - 2.14.3](https://www.openmp.org/spec-html/5.1/openmpsu66.html)) and `target exit data` ([OpenMP 5.1 - 2.14.4](https://www.openmp.org/spec-html/5.1/openmpsu67.html)) are available.
These can be helpful when performing the mapping in separate functions, e.g. in the constructor and destructor of a class.

In [ ]:
%%cpp_omp_target -o code/target-enter-data.cpp

int *vec = new int[10];
for (auto i = 0; i < 10; ++i)
    vec[i] = i;

#pragma omp target enter data map(to: vec[0:10])
                  👆

#pragma omp target teams distribute parallel for
for (auto i = 0; i < 10; ++i)
    vec[i] *= 2;

#pragma omp target exit data map(from: vec[0:10])
                  👆

for (auto i = 0; i < 10; ++i)
    std::cout << vec[i] << " ";
std::cout << std::endl;

free(vec);

For `target enter data`, the map clause must be either `to` or `alloc`.
For `target exit data`, the map clause must be either `from`, `release`, or `delete`.

## Exercise: Revise Stream Benchmark

Your task is to refactor the stream benchmark to use explicit target data regions (or unstructured equivalents).
The code, which you probably altered in the last exercise, is still available at [code/exercise/stream.cpp](code/exercise/stream.cpp).
In case you missed the last exercise, feel free to copy the contents of [code/solution/stream.cpp](code/solution/stream.cpp).

Start the refactoring by adding data mappings to the main loop **only**.

```c++
#pragma omp target ...
for (size_t i = 0; i < nx; ++i)
    dest[i] = src[i] + 1;
```

You can compile and execute the application with the cells below afterwards (note the omitted `-gpu=mem:managed`).

In [ ]:
!nvc++ -O3 -std=c++17 -mp=gpu -Minfo=mp -o code/exercise/stream code/exercise/stream.cpp

In [ ]:
!code/exercise/stream $((64 * 1024 * 1024)) 4

After you have implemented a correct version (`Stream check completed` is printed at the end), evaluate performance and compare it with the original version built on managed memory.

Next, add another target data region around the main iteration loop

```c++
for (size_t i = 0; i < nIt; ++i) {
    ...
}
``` 

In [ ]:
!nvc++ -O3 -std=c++17 -mp=gpu -Minfo=mp -o code/exercise/stream code/solution/stream.cpp

In [ ]:
!code/exercise/stream $((64 * 1024 * 1024)) 4

After you have implemented a correct version (`Stream check completed` is printed at the end), evaluate performance once again and compare it with the two previous versions.

### Possible Solution

A possible solution is given in [code/solution/stream-target-data.cpp](code/solution/stream-target-data.cpp).
It can be compiled and executed using the following cells.

In [ ]:
!nvc++ -O3 -std=c++17 -mp=gpu -Minfo=mp -o code/solution/stream code/solution/stream-target-data.cpp

In [ ]:
!NVCOMPILER_ACC_NOTIFY=1 code/solution/stream $((64 * 1024 * 1024)) 4

## Collapsing Loops

Similar to how loops are handled on CPU, `collapse` can be used to merge multiple loops in a perfect nest.
This is especially important on GPUs since massive parallelism is required to fully utilize the hardware.

In [ ]:
%%cpp_omp_target -o code/collapse.cpp

#pragma omp target teams distribute parallel for simd collapse(2)
                                                      👆
for (auto i = 0; i < 2; ++i)
    for (auto j = 0; j < 5; ++j)
        printf("Team %d, thread %d, i = %d\n", omp_get_team_num(), omp_get_thread_num(), i * 5 + j);

## Reductions



In [ ]:
%%cpp_omp_target -o code/reduction.cpp

auto sum = 0;

#pragma omp target teams distribute parallel for reduction( + : sum )
                                                 👆
for (auto i = 0; i < 100; ++i)
    sum += i;

std::cout << "Sum is " << sum << std::endl;

## Exercise: 2D Stencil

This application solves a 2D finite difference discretization of the Laplace equation using Jacobi iterations.
It can, among many other things, be used to simulate heat distribution.

<img src="https://upload.wikimedia.org/wikipedia/commons/0/01/Heat.gif" alt="heat equation" width="50%"/>

For doing the exercise, understanding the finer details of the algorithm are not important.
In essence, an update for each point of a 2D grid is computed based on the values of neighboring points in a repeating fashion.


**If you have a background in *numerics*:**
This is a *finite volume* or *finite difference* discretization of the heat equation $\frac{\partial u}{\partial t} = \alpha \nabla^2 u$ using *central finite differences* for the spatial derivative (Laplacian) and an *explicit Euler* method for time stepping on a *Cartesian grid*.
Each update computes a new value `uNew[i, j]` from the old field `u[i, j]` and its four nearest neighbors (two per cardinal direction).

**If you have a background in *computer science*:**
This is a *two-fold loop nest* applying a *local stencil operation*.
Each grid point depends only on a *localized neighborhood*—in this case, the 4 cardinal neighbors.
The pattern is usually memory-bound and well-suited for studying data movement, cache behavior, and domain decomposition across GPUs.

**If you have a background in *image processing*:**
This is a *filter* or *convolution*-like operation with a fixed kernel accessing neighboring pixels.
The 'filter' runs repeatedly over time steps, effectively blurring (diffusing) a given image in each iteration.


The serial baseline code is available at [code/exercise/stencil-2d.cpp](code/exercise/stencil-2d.cpp).
It can be compiled and executed using the following cells.

In [ ]:
!g++ -O3 -march=native -std=c++17 -Wall -o code/exercise/stencil-2d code/exercise/stencil-2d.cpp

In [ ]:
!code/exercise/stencil-2d

Optionally, the default parameters can be overwritten with `./stencil-2d num_cells_x_ num_cells_y num_iterations`

In [ ]:
!code/exercise/stencil-2d 1024 1024 64

Your task is to parallelize and port the application to GPU with OpenMP target offloading.
In contrast to the previous exercises, this application does not perform a correctness check.
Instead, it prints a single diagnostic number (the L2 norm of the residual).

For **identical grid sizes and number of iterations**, this number must stay constant.

Start by adding target data mappings and target offloading of the main loop

```c++
for (size_t j = 1; j < ny - 1; ++j) {
    for (size_t i = 1; i < nx - 1; ++i) {
        uNew[j * nx + i] = 0.25 * (  u[j * nx + i - 1] \
                                   + u[j * nx + i + 1] \
                                   + u[(j - 1) * nx + i] \
                                   + u[(j + 1) * nx + i]);
    }
}
```

You can compile and execute the application with the cells below afterwards (add `-gpu=mem:managed` if you want to use managed memory).

In [ ]:
!nvc++ -O3 -std=c++17 -mp=gpu -Minfo=mp -o code/exercise/stencil-2d code/exercise/stencil-2d.cpp

In [ ]:
!code/exercise/stencil-2d $((8 * 1024)) $((8 * 1024)) 64

Compare performance for these scenarios:

* parallelization of the **outer** loop only,
* parallelization of the **inner** loop only, and
* parallelization of **both** loops, e.g. using a `collapse` clause.

Bonus: If you are done quickly, refactor the application once again to also perform the initialization and residual norm computation on GPU to avoid almost all data transfers between CPU and GPU:

```c++
for (size_t j = 0; j < ny; ++j) {
    for (size_t i = 0; i < nx; ++i) {
        if (0 == i || 0 == j || nx - 1 == i || ny - 1 == j) {
            ...
        } else {
            ...
        }
    }
}
```

```c++
double res = 0;
for (size_t j = 1; j < ny - 1; ++j) {
    for (size_t i = 1; i < nx - 1; ++i) {
        const double localRes = 4 * u[j * nx + i] - (u[j * nx + i - 1] + u[j * nx + i + 1] + u[(j - 1) * nx + i] + u[(j + 1) * nx + i]);
        res += localRes * localRes;
    }
}
```

Can you get the application to run at **above 35,000 MLUPS** (around 560 GB/s bandwidth)?

### Possible Solution

A possible solution is given in [code/solution/stencil-2d.cpp](code/solution/stencil-2d.cpp).
It can be compiled and executed using the following cells.

In [ ]:
!nvc++ -O3 -std=c++17 -mp=gpu -o code/solution/stencil-2d code/solution/stencil-2d.cpp

In [ ]:
!code/solution/stencil-2d $((8 * 1024)) $((8 * 1024)) 64